# ECE1513 practical session 3: Intro to Pytorch!
## Objectives
    1. Create torch dataset
    2. Init data loaders
    3. Build fully connected network using Pytorch
    4. Train and test the model



## Import required libraries

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms


## Dataset

The MNIST dataset is a famous dataset in machine learning and computer vision, consisting of 28x28 grayscale images of handwritten digits (0 through 9). Each image is associated with a label indicating the digit it represents.

Here's an introduction to the MNIST dataset along with Python code to load it using PyTorch and visualize some sample images:

In [17]:
# Define a transformation to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the MNIST training and test datasets
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)



## Dataloader

PyTorch DataLoader is an essential component when working with large datasets in deep learning. Here are some reasons why PyTorch DataLoader is important:

* Efficient Data Loading: DataLoader efficiently loads data in batches, allowing you to work with large datasets that may not fit entirely into memory. It loads data in parallel using multiple workers, which can significantly speed up the training process.

* Batch Processing: Deep learning models often train on batches of data rather than individual samples. DataLoader automatically batches the data according to the specified batch size, making it easy to train models with mini-batch gradient descent.

* Shuffling: DataLoader can shuffle the data during training, which is crucial for preventing the model from memorizing the order of the data. Shuffling helps in generalization and improves the model's performance on unseen data.

* Data Transformation: DataLoader allows you to apply data transformations (e.g., normalization, resizing, data augmentation) on the fly. This helps in preprocessing the data before feeding it into the model, enhancing the model's robustness and performance.

* Integration with PyTorch Models: DataLoader seamlessly integrates with PyTorch models and training loops. It provides an iterable interface that can be easily used in training loops to iterate over batches of data.

* Customization: DataLoader is highly customizable and can be configured with various parameters such as batch size, shuffle, number of workers, and drop last batch. This flexibility allows you to tailor the data loading process according to your specific requirements.

In [18]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [19]:
for batch_idx,(data , labels) in enumerate(train_loader):
  print(f'input {batch_idx}: {data.shape}, target: {labels.shape}')
  break



input 0: torch.Size([32, 1, 28, 28]), target: torch.Size([32])


## Visualize random sample from training loader

4     4     4     2


In [20]:
# Function to show images
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

images, labels = next(iter(train_loader))

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(' '.join('%5s' % labels[j].item() for j in range(4)))

## Lets build out network

- Class Name: `NeuralNetwork`
- Inherits from: `torch.nn.Module`
- Initialization:
  - Creates a fully connected layer (`fc1`) with input size of 28*28 and output size of 128 neurons.
  - Applies ReLU activation function (`relu`).
  - Creates another fully connected layer (`fc2`) with input size of 128 and output size of 64 neurons.
  - Creates the final fully connected layer (`fc3`) with input size of 64 and output size of 10 neurons.
- Forward Method:
  - Reshapes the input tensor `x` into a 2D tensor with size (-1, 28 * 28), where -1 indicates batch size.
  - Applies ReLU activation to the output of the first fully connected layer (`fc1`).
  - Applies ReLU activation to the output of the second fully connected layer (`fc2`).
  - Computes the output of the final fully connected layer (`fc3`).
- Returns: The output tensor representing the predicted class scores.


In [21]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = torch.nn.Linear(28 * 28, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 10)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Init the netowrk and choose a loss function and optimizer

- Object Creation:
  - Creates an instance of the `NeuralNetwork` class and assigns it to the variable `net`.
- Loss Function:
  - Defines the loss function as Cross Entropy Loss (`torch.nn.CrossEntropyLoss()`), which is commonly used for multi-class classification tasks.
- Optimizer:
  - Uses Stochastic Gradient Descent (SGD) optimizer (`torch.optim.SGD`) to optimize the parameters of the neural network.
  - Specifies the parameters to be optimized using `net.parameters()`.
  - Sets the learning rate (`lr`) to 0.01 for controlling the step size during optimization.


In [22]:
net = NeuralNetwork()
inputs, targets = next(iter(train_loader))
net(inputs)

tensor([[-0.0631, -0.0476,  0.1192,  0.1045, -0.0029,  0.1912, -0.0410,  0.0335,
          0.1269,  0.0852],
        [-0.0858, -0.0773, -0.0037,  0.0011, -0.0835,  0.1592, -0.0753,  0.0643,
          0.0903,  0.1006],
        [-0.1157, -0.0111,  0.0673,  0.1273, -0.0223,  0.2180, -0.0544,  0.0351,
          0.0104,  0.1272],
        [-0.1047, -0.0359,  0.0203,  0.1103, -0.0480,  0.1310, -0.0605,  0.0840,
          0.0149,  0.1151],
        [-0.1557,  0.0631, -0.0423,  0.0383,  0.0079,  0.2168, -0.0373,  0.0463,
          0.0315,  0.1073],
        [-0.1343, -0.0073,  0.1362,  0.1277, -0.0126,  0.1125, -0.0887,  0.0632,
          0.0358,  0.1256],
        [-0.0625,  0.0561,  0.1440,  0.1043, -0.0234,  0.1158, -0.0833, -0.0180,
          0.0473,  0.0047],
        [-0.1479,  0.0270,  0.0830,  0.1302, -0.0383,  0.1369, -0.0759, -0.0084,
          0.0684,  0.0533],
        [-0.0731, -0.0294,  0.0824,  0.0996,  0.0044,  0.0811, -0.0796,  0.0032,
          0.0474,  0.1225],
        [-0.1235,  

In [23]:
# init loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

## Train the network

- Training Loop:
  - `for epoch in range(5):`: Loops over the dataset for a specified number of epochs (5 in this case).
  - `running_loss = 0.0`: Initializes the running loss variable to zero at the start of each epoch.
  - `for i, data in enumerate(train_loader, 0):`: Loops over the batches of data in the training dataset using `enumerate(train_loader, 0)`.
    - `inputs, labels = data`: Unpacks the input data and labels from the batch.
    - `optimizer.zero_grad()`: Clears the gradients of all optimized parameters before backward pass.
    - `outputs = net(inputs)`: Performs forward pass to get the predicted outputs from the neural network.
    - `loss = criterion(outputs, labels)`: Calculates the loss between the predicted outputs and the ground truth labels using the specified criterion (CrossEntropyLoss).
    - `loss.backward()`: Backpropagates the loss gradient through the network's parameters.
    - `optimizer.step()`: Updates the parameters of the model using the gradients computed in the backward pass.
    - `running_loss += loss.item()`: Accumulates the loss for the current batch.
  - `print('[%d] loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))`: Prints the average loss for the epoch, calculated by dividing the total accumulated loss by the number of batches in the training dataset.
  - `running_loss = 0.0`: Resets the running loss variable to zero for the next epoch.
- After Training:
  - Prints "Finished Training" after completing all epochs.


In [24]:
from sklearn.metrics import f1_score

# Train the neural network
for epoch in range(10):  # Loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    total = 0
    all_predicted = []
    all_labels = []

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # Zero the parameter gradients

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Collect predictions and labels for F1 score calculation
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        running_loss += loss.item()

    # Calculate F1 score
    f1 = f1_score(all_labels, all_predicted, average='weighted')

    # Print the loss, accuracy, and F1 score after each epoch
    print('[%d] loss: %.3f | accuracy: %.2f%% | F1 score: %.4f' % (epoch + 1, running_loss / len(train_loader), 100 * correct / total, f1))
    running_loss = 0.0

print('Finished Training')


[1] loss: 0.699 | accuracy: 80.68% | F1 score: 0.8066
[2] loss: 0.319 | accuracy: 90.55% | F1 score: 0.9052
[3] loss: 0.266 | accuracy: 92.14% | F1 score: 0.9212
[4] loss: 0.225 | accuracy: 93.32% | F1 score: 0.9331
[5] loss: 0.193 | accuracy: 94.33% | F1 score: 0.9432
[6] loss: 0.168 | accuracy: 95.09% | F1 score: 0.9508
[7] loss: 0.147 | accuracy: 95.62% | F1 score: 0.9562
[8] loss: 0.132 | accuracy: 96.11% | F1 score: 0.9611
[9] loss: 0.119 | accuracy: 96.45% | F1 score: 0.9644
[10] loss: 0.109 | accuracy: 96.75% | F1 score: 0.9674
Finished Training


## Test the model

- Testing Loop:
  - `correct = 0`: Initializes a variable to count the number of correctly predicted labels.
  - `total = 0`: Initializes a variable to count the total number of test samples.
  - `with torch.no_grad():`: Context manager to disable gradient computation, since we're only evaluating the model, not training it.
    - `for data in test_loader:`: Loops over the batches of data in the test dataset.
      - `images, labels = data`: Unpacks the input images and labels from the test batch.
      - `outputs = net(images)`: Performs forward pass to get the predicted outputs from the neural network.
      - `_, predicted = torch.max(outputs.data, 1)`: Computes the predicted labels by selecting the index with the maximum value from the output tensor.
      - `total += labels.size(0)`: Increments the total count by the number of labels in the batch.
      - `correct += (predicted == labels).sum().item()`: Increments the correct count by the number of correct predictions in the batch.
  - `print('Accuracy on the 10000 test images: %d %%' % (100 * correct / total))`: Calculates and prints the accuracy of the model on the test dataset as a percentage.


In [25]:
# Test the neural network
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Testing accuracy is: %d %%' % (100 * correct / total))

Testing accuracy is: 96 %
